In [2]:
# Auto-reload changes in src while developing
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

%load_ext autoreload
%autoreload 2
%aimport src.gee_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import ee, geetools
ee.Initialize()

In [39]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.gee_utils import (
    upload_to_gee,
    check_asset_types,
    inspect_asset_metadata,
    count_assets_in_folder,
    create_gee_folder,
    delete_all_subfolders,
    make_assets_public,
    walk_assets,
    make_tree_public,
)

In [ ]:
import os

# Local folder with GeoTIFFs
local_folder = "../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps"

# GEE asset folder
gee_asset_folder = "projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps"

# List all .tif files
tif_files = [f for f in os.listdir(local_folder) if f.endswith(".tif")]

for tif in tif_files[:5]:
    local_path = os.path.join(local_folder, tif)
    upload_to_gee(local_path, gee_asset_folder, execute=False)
djfkl   

Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_2008.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_2008
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_2018.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_2018
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_2019.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_2019
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_1991.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_1991
Source: ../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps/InundationMap_1990.tif
Asset:  projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps/InundationMap_1990


In [44]:
asset_list = [d['id'] for d in 
              ee.data.listAssets("projects/ee-okavango/assets/dsms")['assets']]

asset_list[:6]

['projects/ee-okavango/assets/dsms/34s6807960_20230709_dtm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807965_20230709_ds2_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807965_20230709_dsm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807965_20230709_dtm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807970_20230709_ds2_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807970_20230709_dsm_1_100_egm08_cog']

In [45]:
asset_path = 'projects/ee-okavango/assets/shapes/testing_grounds_20241127'
asset = ee.FeatureCollection(asset_path)

# Get information about the asset
info = asset.getInfo()

info

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'version': 1733849180036364,
 'id': 'projects/ee-okavango/assets/shapes/testing_grounds_20241127',
 'properties': {'system:asset_size': 7369},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[22.969779905690068, -19.103550567786034],
      [22.982804225398247, -19.33262114707843],
      [23.180895834770727, -19.336720261980165],
      [23.17084528571743, -19.2183413491786],
      [23.160818226000984, -19.099959096000124],
      [22.969779905690068, -19.103550567786034]]]},
   'id': '00000000000000000000',
   'properties': {}}]}

In [ ]:

# def set_folder_permissions(folder_path, email, role="READER"):
#     # List all assets in the folder
#     asset_list = ee.data.listAssets({'parent': folder_path})['assets']
    

#     for asset in asset_list:
#         asset_id = asset['id']
#         # Get current access control list (ACL) for the asset
#         acl = ee.data.getAssetAcl(asset_id)
        
#         # Update ACL to share with the new user
#         acl['owners'] = acl.get('owners', [])
#         acl['writers'] = acl.get('writers', [])
#         acl['readers'] = acl.get('readers', [])
#         print (acl['readers'] )
#         if role == "READER":
#             acl['readers'].append(f"user:{email}")  # Add the user prefix
#         elif role == "WRITER":
#             acl['writers'].append(f"user:{email}")  # Add the user prefix
        
        
#         # Set the new ACL for the asset
#         ee.data.setAssetAcl(asset_id, acl)

# # Example usage
# folder_path = "projects/ee-quatratavia/assets/berms/DEP3"
# user_email = 'aaronkoop@gmail.com'  # Email of the user you want to share with
# set_folder_permissions(folder_path, user_email, role="READER")  # Role can be "READER" or "WRITER"


In [ ]:
# Code to move assets around
src = ee.Asset("projects/ee-quatratavia/assets/monthly_LST")
dst = ee.Asset("projects/ee-quatratavia/assets/CA_fire/monthly_LST")

# src.move(dst)


ee.Asset('projects/ee-quatratavia/assets/CA_fire/monthly_LST')

In [ ]:
src = ee.Asset("projects/ee-quatratavia/assets/MBTS")
dst = ee.Asset("projects/ee-quatratavia/assets/CA_fire/MBTS")

# src.move(dst)


ee.Asset('projects/ee-quatratavia/assets/CA_fire/MBTS')

In [ ]:
# TODO - fix this or remove.  currently deletes everything
# Define the polygon by loading the FeatureCollection
polygon_fc = ee.FeatureCollection("projects/ee-okavango/assets/shapes/testing_grounds_20241127").first()
polygon_geometry = polygon_fc.geometry()

polygon_geometry = ee.Geometry.Polygon(
        [[[21.716601878680233, -17.986800918676114],
          [21.716601878680233, -18.198273997193347],
          [21.988513499773983, -18.198273997193347],
          [21.988513499773983, -17.986800918676114]]])
# Define the folder containing the images
folder_path = "projects/ee-okavango/assets/dsms"

# List all assets in the folder
asset_list = ee.data.listAssets({'parent': folder_path})['assets']

# Function to check if an image is outside the polygon
def is_image_outside_polygon(image_path, polygon_geometry):
    try:
        image = ee.Image(image_path)
        image_geometry = image.geometry()
        return not polygon_geometry.contains(image_geometry, 1).getInfo()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return False

# Loop through assets and delete images outside the polygon
for asset in asset_list:
    asset_path = asset['name']

    if is_image_outside_polygon(asset_path, polygon_geometry):
        print(f"Deleting image: {asset_path}")
        ee.data.deleteAsset(asset_path)
    else:
        print(f"Keeping image: {asset_path}")


In [ ]:
# Share all assets under a root with a specific user (dry-run first)
from src.gee_utils import share_tree_with_user

ROOT = "projects/ee-okavango/assets/dsms"
EMAIL = "user@example.com"  # or use 'group:my-group@domain' for groups

# Dry run: prints what would change
share_tree_with_user(ROOT, EMAIL, role='READER', dry_run=True)

# When ready to apply, set dry_run=False
# share_tree_with_user(ROOT, EMAIL, role='READER', dry_run=False, sleep_sec=0.1)
